# Data Mining Project - WNBA Playoffs Prediction - G24

## Business Understanding

#### Our data
Basketball tournaments are usually split in two parts. First, all teams play each other aiming to achieve the greatest number of wins possible. Then, at the end of the first part of the season, a pre determined number of teams which were able to win the most games are qualified to the playoff season, where they play series of knock-out matches for the trophy.

For the 10 years, data from players, teams, coaches, games and several other metrics were gathered and arranged on this dataset. The goal is to use this data to predict which teams will qualify for the playoffs in the next season.



#### Competition Format
The 12 teams in the WNBA are split into an Eastern Conference and a Western Conference. WNBA fixtures begin with preseason games in May before each team plays 20 home games and 20 road games during the regular season.

The aim for every team is to qualify for the Playoffs, which begin in September each year.

The WNBA teams with the eight best regular season records regardless of standing qualify for the Playoffs. Higher seeds matchup with lower seeds, so the top seed faces the eight seed, the second seed faces the seven seed and so on.

When it comes to betting on the Playoffs, the first round are best-of-three series. The semifinals and final are both best-of-five, meaning WNBA teams need to record three wins to claim victory in the series.

## Database Connection

We used a free service to host our database. The Database is in PostgreSQL.

In [ ]:
# DB Credentials
import json

with open("config.json") as config_file:
    config = json.load(config_file)

host = config["db_host"]
user = config["db_user"]
password = config["db_password"]
database = config["db_database"]
schema = config["db_schema"]

In [ ]:
import psycopg2
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

connection = psycopg2.connect(
    host=host,
    user=user,
    password=password,
    database=database
)

cursor = connection.cursor()

def execute(query):
    cursor.execute(query)
    connection.commit()
    return cursor.fetchall()

def fetch(query):
    cursor.execute(query)
    return cursor.fetchall()

SELECT = "SELECT * FROM " + schema + "." # + table_name 
INSERT = "INSERT INTO " + schema + "." # + table_name + " VALUES " + values
UPDATE = "UPDATE " + schema + "." # + table_name + " SET " + column_name + " = " + value
DELETE = "DELETE FROM " + schema + "."  # + table_name + " WHERE " + column_name + " = " + value

 The data about the players, teams and coaches consist of following relations:

    awards_players (96 objects) - each record describes awards and prizes received by players across 10 seasons,
    coaches (163 objects) - each record describes all coaches who've managed the teams during the time period,
    players (894 objects) - each record contains details of all players,
    players_teams (1877 objects) - each record describes the performance of each player for each team they played,
    series_post (71 objects) - each record describes the series' results,
    teams (143 objects) - each record describes the performance of the teams for each season,
    teams_post (81 objects) - each record describes the results of each team at the post-season.


In [ ]:
awards_players = fetch(SELECT + "awards_players") # awards and prizes received by players across 10 seasons,
coaches = fetch(SELECT + "coaches") # all coaches who've managed the teams during the time period,
players = fetch(SELECT + "players") # details of all players,
players_teams = fetch(SELECT + "players_teams") # performance of each player for each team they played,
series_post = fetch(SELECT + "series_post") # series' results,
teams = fetch(SELECT + "teams") # performance of the teams for each season,
teams_post = fetch(SELECT + "teams_post") # results of each team at the post-season.

## Data Understanding

In [ ]:
#print(coaches.description)

for coach in coaches:
    print(coach)
    

## Process Coaches ------ para cada um verificar nulos e tipos!!!!!

In [ ]:

# coachID;year;tmID;lgID;stint;won;lost;post_wins;post_losses
# mudar BD diretamente??
coaches_df = pd.DataFrame(coaches, columns=['coachID', 'year', 'tmID', 'lgID', 'stint', 'won', 'lost', 'post_wins', 'post_losses'])
coaches_df.head()

# drop columns 'lgID'
coaches_df.drop('lgID', axis=1, inplace=True)

# mergir totais de vitórias e derrotas??

## Process Awards Players

In [ ]:
# drop lgID
# playerID;award;year;lgID
awards_players_df = pd.DataFrame(awards_players, columns=['playerID', 'award', 'year', 'lgID'])

awards_players_df.drop('lgID', axis=1, inplace=True)

# contar awards por playerID?


## Process Players

In [ ]:
# drop firstseason, lastseason, college, collegeOther

#bioID;pos;firstseason;lastseason;height;weight;college;collegeOther;birthDate;deathDate
players_df = pd.DataFrame(players, columns=['bioID', 'pos', 'firstseason', 'lastseason', 'height', 'weight', 'college', 'collegeOther', 'birthDate', 'deathDate'])

players_df.drop(['firstseason', 'lastseason', 'college', 'collegeOther'], axis=1, inplace=True)

# drop rows where deathDate is different from 0000-00-00
players_df = players_df[players_df['deathDate'] == '0000-00-00']

# drop deathDate
# players_df.drop('deathDate', axis=1, inplace=True)
# 
# a posição é relevante??
players_df.head()


players_attributes = players_df
correlation_matrix = players_attributes.corr()

sorted_correlations = correlation_matrix.unstack().sort_values(ascending=False)

plt.figure(figsize=(20, 20))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm')
plt.show()

# Get the pairs of attributes with the highest correlation values
most_correlated_pairs = sorted_correlations[sorted_correlations > 0.80]
most_correlated_pairs = most_correlated_pairs[most_correlated_pairs < 1.0]


#delete repeated pairs (e.g. (a,b) and (b,a))
most_correlated_pairs = most_correlated_pairs[::2]
print(most_correlated_pairs)

## Process Players Teams

In [ ]:
# playerID;year;stint;tmID;lgID;GP;GS;minutes;points;oRebounds;dRebounds;rebounds;assists;steals;blocks;turnovers;PF;fgAttempted;fgMade;ftAttempted;ftMade;threeAttempted;threeMade;dq;PostGP;PostGS;PostMinutes;PostPoints;PostoRebounds;PostdRebounds;PostRebounds;PostAssists;PostSteals;PostBlocks;PostTurnovers;PostPF;PostfgAttempted;PostfgMade;PostftAttempted;PostftMade;PostthreeAttempted;PostthreeMade;PostDQ

players_teams_df = pd.DataFrame(players_teams, columns=['playerID', 'year', 'stint', 'tmID', 'lgID', 'GP', 'GS', 'minutes', 'points', 'oRebounds', 'dRebounds', 'rebounds', 'assists', 'steals', 'blocks', 'turnovers', 'PF', 'fgAttempted', 'fgMade', 'ftAttempted', 'ftMade', 'threeAttempted', 'threeMade', 'dq', 'PostGP', 'PostGS', 'PostMinutes', 'PostPoints', 'PostoRebounds', 'PostdRebounds', 'PostRebounds', 'PostAssists', 'PostSteals', 'PostBlocks', 'PostTurnovers', 'PostPF', 'PostfgAttempted', 'PostfgMade', 'PostftAttempted', 'PostftMade', 'PostthreeAttempted', 'PostthreeMade', 'PostDQ'])

# drop lgID
players_teams_df.drop('lgID', axis=1, inplace=True)

# ????

players_teams_df.head()


## Process Series Post

In [ ]:
# year;round;series;tmIDWinner;lgIDWinner;tmIDLoser;lgIDLoser;W;L
series_post_df = pd.DataFrame(series_post, columns=['year', 'round', 'series', 'tmIDWinner', 'lgIDWinner', 'tmIDLoser', 'lgIDLoser', 'W', 'L'])

# drop lgIDWinner, lgIDLoser

series_post_df.drop(['lgIDWinner', 'lgIDLoser'], axis=1, inplace=True)

# qualificar o round??

series_post_df.head()

## Process Teams

In [ ]:
# year;lgID;tmID;franchID;confID;divID;rank;playoff;seeded;firstRound;semis;finals;name;o_fgm;o_fga;o_ftm;o_fta;o_3pm;o_3pa;o_oreb;o_dreb;o_reb;o_asts;o_pf;o_stl;o_to;o_blk;o_pts;d_fgm;d_fga;d_ftm;d_fta;d_3pm;d_3pa;d_oreb;d_dreb;d_reb;d_asts;d_pf;d_stl;d_to;d_blk;d_pts;tmORB;tmDRB;tmTRB;opptmORB;opptmDRB;opptmTRB;won;lost;GP;homeW;homeL;awayW;awayL;confW;confL;min;attend;arena

teams_df = pd.DataFrame(teams, columns=['year', 'lgID', 'tmID', 'franchID', 'confID', 'divID', 'rank', 'playoff', 'seeded', 'firstRound', 'semis', 'finals', 'name', 'o_fgm', 'o_fga', 'o_ftm', 'o_fta', 'o_3pm', 'o_3pa', 'o_oreb', 'o_dreb', 'o_reb', 'o_asts', 'o_pf', 'o_stl', 'o_to', 'o_blk', 'o_pts', 'd_fgm', 'd_fga', 'd_ftm', 'd_fta', 'd_3pm', 'd_3pa', 'd_oreb', 'd_dreb', 'd_reb', 'd_asts', 'd_pf', 'd_stl', 'd_to', 'd_blk', 'd_pts', 'tmORB', 'tmDRB', 'tmTRB', 'opptmORB', 'opptmDRB', 'opptmTRB', 'won', 'lost', 'GP', 'homeW', 'homeL', 'awayW', 'awayL', 'confW', 'confL', 'min', 'attend', 'arena'])

# drop lgID, divID, seeded, arena? attend?, tmORB, tmDRB, tmTRB, opptmORB, opptmDRB, opptmTRB,

teams_df.drop(['lgID', 'divID', 'seeded', 'arena', 'attend', 'tmORB', 'tmDRB', 'tmTRB', 'opptmORB', 'opptmDRB', 'opptmTRB'], axis=1, inplace=True)


teams_df.head()

## Process Teams Post

In [ ]:
# year;tmID;lgID;W;L

teams_post_df = pd.DataFrame(teams_post, columns=['year', 'tmID', 'lgID', 'W', 'L'])

# drop lgID

teams_post_df.drop('lgID', axis=1, inplace=True)

In [ ]:
connection.close()